In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [11]:
# Upload the data and conduct preprocessing.

data = pd.read_csv("h1b_kaggle.csv")
data = data.dropna(axis=0, how='any')

data = data[~data.CASE_STATUS.str.contains('WITHDRAWN')]
data = data[~data.CASE_STATUS.str.contains('WITHDRAWN')]
data = data[~data.CASE_STATUS.str.contains('ASSIGNED')]
data = data[~data.CASE_STATUS.str.contains('INVALIDATED')]
data = data.replace(to_replace="CERTIFIED", value=1)
data = data.replace(to_replace="DENIED", value=0)
data = data.replace(to_replace="REJECTED", value=0)
data = data.drop('lon', 1)
data = data.drop('lat', 1)
data = data.drop('JOB_TITLE', 1)
data = data.drop('Unnamed: 0', 1)
data['FULL_TIME_POSITION'].replace(['Y','N'],['1','0'],inplace=True)


# Express worksite based only on states, and give them numbers.
Cities,States = data['WORKSITE'].str.split(', ',1).str
city_to_state = dict(zip(data['WORKSITE'], States))
data['WORKSITE']= data['WORKSITE'].map(city_to_state)
data = data[~data.WORKSITE.str.match('NA')]
data.WORKSITE = pd.Categorical(data.WORKSITE)
data['WORKSITE'] = data.WORKSITE.cat.codes

#Quantize the wages and get rid of outliers
temp=data.copy()
temp["PREVAILING_WAGE"].hist(bins=200,range=[0,400000])
t=temp[np.abs(temp["PREVAILING_WAGE"]-temp["PREVAILING_WAGE"].mean())<=(0.1*temp["PREVAILING_WAGE"].std())]
t=t[temp["PREVAILING_WAGE"]>0]
b=pd.qcut(t["PREVAILING_WAGE"],200)
for i in range(1,t["PREVAILING_WAGE"].size):
    t["PREVAILING_WAGE"].values[i]=b.values[i].mid
data=t

# Create a feature for number of applications per company.
lookup = data.EMPLOYER_NAME.value_counts()
data['APPS_PER_COMPANY']= data['EMPLOYER_NAME'].map(lookup)

# Get rid of the companies with less than 4 applications.
data=data[data["APPS_PER_COMPANY"]>3]

#Clean up SOC_NAME metadata.
data['SOC_NAME'] = data['SOC_NAME'].str.upper()
data['SOC_NAME'] = data['SOC_NAME'].map(lambda x: str(x).lstrip('*').rstrip('*'))
data.SOC_NAME = data.SOC_NAME.str.replace('[^a-zA-Z]', '')
data.SOC_NAME = data.SOC_NAME.str.replace('ALLOTHER', '')

# Get rid of SOC's that have less than 4 appearances in data.
# Create a feature for number of applications per SOC type.
lookup = data.SOC_NAME.value_counts()
data['APPS_PER_SOC']= data['SOC_NAME'].map(lookup)
data=data[data["APPS_PER_SOC"]>3]


# Create a feature for number of applications per worksite.
lookup = data.WORKSITE.value_counts()
data['APPS_PER_WORKSITE']= data['WORKSITE'].map(lookup)

# Convert SOC and EMPLOYER_NAME to numeric values.
data.SOC_NAME = pd.Categorical(data.SOC_NAME)
data['SOC_NAME'] = data.SOC_NAME.cat.codes
data.EMPLOYER_NAME = pd.Categorical(data.EMPLOYER_NAME)
data['EMPLOYER_NAME'] = data.EMPLOYER_NAME.cat.codes

# Create a feature for COMPANY_SUCCESS RATE
lookup = data.EMPLOYER_NAME.value_counts()
lookup2 = dict(zip(data.EMPLOYER_NAME.unique(),np.zeros((data.EMPLOYER_NAME.value_counts().size))))
lkp=lookup.size
dd=data.copy()
for i in range(0,lkp):
#    t = time.time()
    loc=dd.EMPLOYER_NAME.values==lookup.index[i]
    lookup2[lookup.index[i]]=(np.sum(dd[loc].CASE_STATUS.values))/lookup.values[i]
    dd=dd[np.invert(loc)]
data['COMPANY_SUCCESS_RATE']= data['EMPLOYER_NAME'].map(lookup2)

# Create a feature for SOC_SUCCESS_RATE
lookup = data.SOC_NAME.value_counts()
lookup2 = dict(zip(data.SOC_NAME.unique(),np.zeros((data.SOC_NAME.value_counts().size))))
lkp=lookup.size
dd=data.copy()
for i in range(0,lkp):
#    t = time.time()
    loc=dd.SOC_NAME.values==lookup.index[i]
    lookup2[lookup.index[i]]=(np.sum(dd[loc].CASE_STATUS.values))/lookup.values[i]
    dd=dd[np.invert(loc)]
data['SOC_SUCCESS_RATE']= data['SOC_NAME'].map(lookup2)

# Create a feature for WORKSITE_SUCCESS_RATE
lookup = data.WORKSITE.value_counts()
lookup2 = dict(zip(data.WORKSITE.unique(),np.zeros((data.WORKSITE.value_counts().size))))
lkp=lookup.size
dd=data.copy()
for i in range(0,lkp):
    loc=dd.WORKSITE.values==lookup.index[i]
    lookup2[lookup.index[i]]=(np.sum(dd[loc].CASE_STATUS.values))/lookup.values[i]
    dd=dd[np.invert(loc)]
data['WORKSITE_SUCCESS_RATE']= data['WORKSITE'].map(lookup2)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [12]:


# Learn the distribution of the data.
data['CASE_STATUS'].value_counts()
# 1    2296865
# 0      54177

data.apply(pd.Series.nunique)
# CASE_STATUS                  2
# EMPLOYER_NAME            52535
# SOC_NAME                   709
# FULL_TIME_POSITION           2
# PREVAILING_WAGE          49077
# YEAR                         6
# WORKSITE                    52
# APPS_PER_COMPANY           877
# APPS_PER_SOC               379
# APPS_PER_WORKSITE           52
# COMPANY_SUCCESS_RATE      1856
# SOC_SUCCESS_RATE           444
# WORKSITE_SUCCESS_RATE       52

# Save the preprocessed data to a csv file
data.to_csv('h1b_test_processed.csv')



In [13]:
data['CASE_STATUS'].value_counts()

1    2296622
0      52519
Name: CASE_STATUS, dtype: int64

In [14]:
data.apply(pd.Series.nunique)

CASE_STATUS                  2
EMPLOYER_NAME            52461
SOC_NAME                   709
FULL_TIME_POSITION           2
PREVAILING_WAGE            200
YEAR                         6
WORKSITE                    52
APPS_PER_COMPANY           888
APPS_PER_SOC               376
APPS_PER_WORKSITE           52
COMPANY_SUCCESS_RATE      1825
SOC_SUCCESS_RATE           445
WORKSITE_SUCCESS_RATE       52
dtype: int64

In [15]:

data = pd.read_csv("h1b_test_processed.csv")
labels = data.CASE_STATUS
data = data.drop('CASE_STATUS', 1)
data = data.drop('EMPLOYER_NAME', 1)
data = data.drop('SOC_NAME', 1)
data = data.drop('Unnamed: 0', 1)
data = data.drop('YEAR', 1)
data = data.drop('WORKSITE', 1)

In [16]:


from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

# Normalize the data to zero mean and unit variance.
scaler = preprocessing.StandardScaler()
data[data.columns] = scaler.fit_transform(data[data.columns])
# data.mean(axis=0)
# data.std(axis=0)



In [27]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [18]:
X_test['CASE_STATUS'] = y_test
X_test_1 = X_test[X_test['CASE_STATUS']==1]
X_test_1_sampled = X_test_1.sample(n=11000)
X_test_0 = X_test[X_test['CASE_STATUS']==0]
X_test = X_test_1_sampled.append(X_test_0)
y_test = X_test['CASE_STATUS']
X_test = X_test.drop('CASE_STATUS', 1)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:


from sklearn.kernel_approximation import RBFSampler
clf = SGDClassifier(loss="log",learning_rate='invscaling', eta0 =0.5, penalty="l1",power_t=0.1, max_iter=100)
rbf_feature = RBFSampler(gamma=0.1, random_state=1)
X_features = rbf_feature.fit_transform(X_train)
clf.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.5, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='log', max_iter=100, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.1, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [20]:
import sklearn.metrics as skm
skm.roc_auc_score(y_test,clf.predict_proba(X_test)[:, 1])

0.71905793243466976

In [21]:
skm.f1_score(y_test,clf.predict(X_test))

0.67832146270773597

In [22]:
print(skm.classification_report(y_test, clf.predict(X_test)))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00     10433
          1       0.51      1.00      0.68     11000

avg / total       0.26      0.51      0.35     21433



/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [23]:
import pandas as pd
pd.crosstab(y_test, clf.predict(X_test), rownames=['true'], colnames=['predict'])

predict,1
true,
0,10433
1,11000


In [24]:
print(clf.score(X_test, y_test))

0.513227266365


In [25]:
data.head()

,FULL_TIME_POSITION,PREVAILING_WAGE,APPS_PER_COMPANY,APPS_PER_SOC,APPS_PER_WORKSITE,COMPANY_SUCCESS_RATE,SOC_SUCCESS_RATE,WORKSITE_SUCCESS_RATE
0,0.389538,11.108846,-0.406269,-1.178385,-0.699124,0.373399,-5.749734,-0.234971
1,0.389538,11.108846,-0.406269,-1.178385,1.839904,0.373399,-5.749734,-0.131467
2,0.389538,11.108846,-0.406045,-1.178385,-0.286782,0.373399,-5.749734,0.666689
3,0.389538,2.960829,-0.406269,-1.178385,-0.687817,0.373399,-5.749734,-1.520779
4,0.389538,1.734477,-0.405949,-1.178385,-0.286782,-0.670470,-5.749734,0.666689


In [26]:
X_test_0.size

93897